In [2]:
import numpy as np
import matplotlib.pyplot as plt
from keras import models
from keras.layers import Dense, Conv1D, Reshape, Flatten, Lambda
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
import keras

Using TensorFlow backend.


In [3]:
class Data:
    def __init__(self, mu, sigma, ni_D):
        self.mu = mu
        self.sigma = sigma
        self.ni_D = ni_D
        #self.real_sample = lambda n_batch: np.random.normal(mu, sigma, (n_batch, ni_D))
        #self.in_sample = lambda n_batch: np.random.rand(n_batch, ni_D)

    def real_sample(self, n_batch):
        return np.random.normal(self.mu, self.sigma, (n_batch, self.ni_D))

    def in_sample(self, n_batch):
        return np.random.rand(n_batch, self.ni_D)


In [60]:
class Machine:
    def __init__(self, n_batch=10, ni_D=100):
        data_mean = 4
        data_stddev = 1.25

        self.n_iter_D = 5
        self.n_iter_G = 5

        self.data = Data(data_mean, data_stddev, ni_D)
        self.gan = GAN(ni_D=ni_D, nh_D=50, nh_G=50)
        self.n_batch = n_batch

    def train_D(self):
        gan = self.gan
        n_batch = self.n_batch
        data = self.data
        Real = data.real_sample(n_batch)
        Z = data.in_sample(n_batch)

        Gen = gan.G.predict(Z)
        gan.D.trainable = True
        print('train_D')
        gan.D_train_on_batch(Real, Gen)

    def train_GD(self):
        gan = self.gan
        n_batch = self.n_batch
        data = self.data
        Z = data.in_sample(n_batch)

        gan.D.trainable = False
        print('train_GD')
        gan.GD_train_on_batch(Z)

    def train_each(self, epoch):
        print('train_each - 0:', epoch)
        for it in range(self.n_iter_D):
            print('train_each - 1:', epoch)
            self.train_D()
        for it in range(self.n_iter_G):
            print('train_each - 2:', epoch)
            self.train_GD()

    def train(self, epochs):
        for epoch in range(epochs):
            self.train_each(epoch)

    def test(self, n_test):
        gan = self.gan
        data = self.data
        Z = data.in_sample(n_test)
        Gen = gan.G.predict(Z)
        return Gen, Z

    def show_hist(self, Real, Gen, Z):
        plt.hist(Real.reshape(-1), histtype='step', label='Real')
        plt.hist(Gen.reshape(-1), histtype='step', label='Generated')
        plt.hist(Z.reshape(-1), histtype='step', label='Input')
        plt.legend(loc=0)

    def test_and_show(self, n_test):
        data = self.data
        Gen, Z = self.test(n_test)
        Real = data.real_sample(n_test)
        self.show_hist(Real, Gen, Z)
        self.print_stat(Real, Gen)

    def run_epochs(self, epochs, n_test):
        self.train(epochs)
        self.test_and_show(n_test)

    def run(self, n_repeat=200, n_show=200, n_test=100):
        for ii in range(n_repeat):
            print('Stage', ii, '(Epoch: {})'.format(ii * n_show))
            self.run_epochs(n_show, n_test)
            plt.show()

    def print_stat(self, Real, Gen):
        def stat(d):
            return (np.mean(d), np.std(d))
        print('Mean and Std of Real:', stat(Real))
        print('Mean and Std of Gen:', stat(Gen))


In [61]:
def add_decorate(x):
    m = K.mean(x, axis=-1, keepdims=True)
    d = K.square(x - m)
    return K.concatenate([x,d], axis=-1)

def add_decorate_shape(input_shape):
    shape = list(input_shape)
    assert len(input_shape) == 2
    shape[1] *= 2
    return tuple(shape)

lr = 2e-4
adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999)

def model_compile(model):
    return model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [62]:
class GAN:
    def __init__(self, ni_D, nh_D, nh_G):
        self.ni_D = ni_D
        self.nh_D = nh_D
        self.nh_G = nh_G
        
        self.D = self.gen_D()
        self.G = self.gen_G()
        self.GD = self.make_GD()
    
    def gen_D(self):
        ni_D = self.ni_D
        nh_D = self.nh_D
        
        D = models.Sequential()
        D.add(Lambda(add_decorate, output_shape=add_decorate_shape, input_shape=(ni_D,)))
        
        D.add(Dense(nh_D, activation='relu'))
        D.add(Dense(nh_D, activation='relu'))
        D.add(Dense(1, activation='sigmoid'))
        
        model_compile(D)

        print('D.summary()')
        D.summary()
        return D
    
    def gen_G(self):
        ni_D = self.ni_D
        nh_G = self.nh_G
        
        G = models.Sequential()
        G.add(Reshape((ni_D, 1), input_shape=(ni_D,)))
        G.add(Conv1D(nh_G, 1, activation='relu'))
        G.add(Conv1D(nh_G, 1, activation='sigmoid'))
        G.add(Conv1D(1, 1))
        G.add(Flatten())
        
        model_compile(G)
        
        print('G.summary()')
        G.summary()
        return G
    
    def make_GD(self):
        G, D = self.G, self.D
        GD = models.Sequential()
        GD.add(G)
        GD.add(D)
        D.trainable = False
        model_compile(GD)
        print('GD.summary()')
        GD.summary()
        D.trainable = True
        return GD
    
    def D_train_on_batch(self, Real, Gen):
        D = self.D
        X = np.concatenate([Real, Gen], axis=0)
        y = np.array([1]*Real.shape[0] + [0]*Gen.shape[0])
        
        print('X.shape',X.shape)
        print('y.shape',y.shape)
        print(X)        
        print(y)
        D.train_on_batch(X, y)
#        D.fit(X,y)
        
    def GD_train_on_batch(self, Z):
        GD = self.GD
        y = np.array([1]*Z.shape[0])
        print('Z.shape',Z.shape)
        print('y.shape',y.shape)
        print(Z)        
        print(y)
        
        
        GD.train_on_batch(Z, y)
#        GD.fit(Z, y)

In [63]:
#import tensorflow.python.keras.backend as K
#sess = K.get_session()
#sess.run(tf.global_variables_initializer())
#keras.backend.get_session().run(tf.local_variables_initializer())
#keras.backend.get_session().run(tf.global_variables_initializer())
machine = Machine(n_batch=1, ni_D=100)
machine.run(n_repeat=200, n_show=200, n_test=100)

D.summary()
Model: "sequential_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_22 (Lambda)           (None, 200)               0         
_________________________________________________________________
dense_64 (Dense)             (None, 50)                10050     
_________________________________________________________________
dense_65 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 51        
Total params: 12,651
Trainable params: 12,651
Non-trainable params: 0
_________________________________________________________________
G.summary()
Model: "sequential_64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_22 (Reshape)         (None, 100, 1)            0         
__

train_each - 2: 0
train_GD
Z.shape (1, 100)
y.shape (1,)
[[0.31711709 0.74666768 0.97363673 0.00320784 0.20649712 0.43529773
  0.35516367 0.8834838  0.05092928 0.47056748 0.97554354 0.67588327
  0.62069579 0.16823331 0.77320504 0.42181072 0.19920323 0.87971047
  0.14394503 0.47170372 0.98918053 0.35690732 0.42270582 0.68573933
  0.78321422 0.69967295 0.59619073 0.27787997 0.64143807 0.7138541
  0.70944069 0.77776561 0.91154834 0.71301071 0.37558076 0.18011807
  0.16325185 0.54391956 0.83036362 0.19562104 0.20833859 0.75307613
  0.80971478 0.7039794  0.34954873 0.90831983 0.07563482 0.28393561
  0.30801281 0.26389502 0.37789938 0.57022039 0.9467839  0.49223517
  0.72843725 0.44908633 0.1374931  0.02886049 0.07748551 0.62544969
  0.49912183 0.70511031 0.32261653 0.70489583 0.18180397 0.25096651
  0.72268501 0.73422162 0.55191547 0.67130565 0.11017153 0.26564373
  0.95462923 0.42430039 0.7488061  0.042625   0.00767703 0.6216177
  0.99385048 0.63471663 0.64332048 0.99775231 0.60575355 0.72

FailedPreconditionError:  Error while reading resource variable _AnonymousVar935 from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/_AnonymousVar935/N10tensorflow3VarE does not exist.
	 [[node mul_884/ReadVariableOp (defined at /home/inswave/anaconda3/envs/udemy/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_209272]

Function call stack:
keras_scratch_graph
